In [1]:
%cd ..

import builtins
builtins.uclresearch_topic = 'GIVENCHY' # 226984 entires
# builtins.uclresearch_topic = 'HAWKING' # 4828104 entries
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config

import smtplib
import base64
from email.mime.text import MIMEText
from email.header import Header
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.utils import COMMASPACE
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
import os
import os.path
import math
import sys
import csv
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import scipy.integrate as spi
import pylab as pl
import networkx as nx
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
from IPython.display import display

/home/ec2-user/May


/home/ec2-user/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


('Reading config file from location: '
 '/home/ec2-user/May/configuration/env.properties')
{'calculate': {'analysis': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'crawl': {'followers': '/home/ec2-user/May/followers',
                    'friends': '/home/ec2-user/May/friends'},
          'cwd': '/home/ec2-user/May/data/givenchy',
          'ml': '/home/ec2-user/May/data/givenchy/pickle',
          'networkx': {'all': '/home/ec2-user/May/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/home/ec2-user/May/data/givenchy/pickle/networkx_friends.dat',
                       'potential': '/home/ec2-user/May/data/givenchy/pickle/networkx_potential.dat'},
         

In [2]:
n_bytes = 2**31
max_bytes = 2**31 - 1

path = "/home/ec2-user/May/data/givenchy/pickle"  

def write_to_pickle(file_path, data):

    bytes_out = pickle.dumps(data)
    with open(file_path, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

def load_pickle(file_path):
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    return pickle.loads(bytes_in)

def loadModel(self):
    read_file = "data/" + self.model_name + ".dat"
    self.model = joblib.load(read_file)
    return self.model


# Load data

In [3]:
model = joblib.load(path+"/xgb_model_GIVENCHY (2).dat")
unique_users = load_pickle(path+"/80000users_3.pkl")
network = load_pickle(path+"/80000users_3.pkl")
users=load_pickle(path+'/Stanford_features.pkl')
seeds = load_pickle(path+'/Stanford_seeds.pkl')

In [4]:
unique_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70013 entries, 0 to 70012
Data columns (total 14 columns):
user_favourites_count               70013 non-null int64
followers_count                     70013 non-null int64
friends_count                       70013 non-null int64
user_id                             70013 non-null int64
user_listed_count                   70013 non-null int64
user_statuses_count                 70013 non-null int64
source_candidates                   70013 non-null object
infec_candidates                    70013 non-null object
user_created_days                   70013 non-null int64
normalized_user_statuses_count      70013 non-null float64
normalized_user_followers_count     70013 non-null float64
normalized_user_favourites_count    70013 non-null float64
normalized_user_listed_count        70013 non-null float64
normalized_user_friends_count       70013 non-null float64
dtypes: float64(5), int64(7), object(2)
memory usage: 7.5+ MB


In [5]:
# ######## infec_candidates ########
# temp=[]
# infec = [temp for i in range(len(unique_users))]
# unique_users["infec_candidates"]= None
# record_num = int(network.describe().iloc[0,0])

# for i in range(record_num):
#     source_candidates = unique_users.loc[i,'source_candidates']
#     #users.loc[i,"infec_candidates"] = list()
#     for j in range(len(source_candidates)):
#         candidates = source_candidates[j]
#         if infec[candidates] == []:
#             infec[candidates]=[i]
#         else :
#             infec[candidates]+=[i]

# unique_users.loc[:,"infec_candidates"]=infec

# unique_users.head()

# reprocess_data

In [6]:
def reprocess_data(time,origins,infectors,users,unique_users):
        i= int(users[users.user_id.isin([origins])].index.values)
        j= int(users[users.user_id.isin([infectors])].index.values)
        source_candidates = unique_users.loc[j,'source_candidates']
        sources = [x for x in source_candidates if users.loc[x,'TwM_time']<=time]
        sources_time=users[users.index.isin(sources)].sort_values(by=['TwM_time'])
        sources_dataframe = unique_users.loc[sources]
        timeList =  [users.loc[x].TwM_tCurrent - users.loc[x].TwM_time for x in sources]
        degreeList = list(unique_users.loc[k, 'followers_count'] + unique_users.loc[k, 'friends_count']  for k in sources)
    
            
        m=sources_time.index.values[0]
        n=sources_time.index.values[-1]
        
        users.loc[j,'UsM_deltaDays0'] = users.loc[m,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount0'] = users.loc[m,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount0'] = users.loc[m,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount0'] = users.loc[m,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount0']=users.loc[m,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount0']=users.loc[m,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount0']=users.loc[m,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount0']=users.loc[m,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount0']=users.loc[m,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount0']=users.loc[m,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount0']=users.loc[m,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'UsM_deltaDays-1'] = users.loc[n,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount-1'] = users.loc[n,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount-1']=users.loc[n,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount-1']=users.loc[n,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount-1']=users.loc[n,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount-1']=users.loc[n,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount-1']=users.loc[n,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount-1']=users.loc[n,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount-1']=users.loc[n,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount-1']=users.loc[n,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount-1']=users.loc[n,'UsM_normalizedUserFriendsCount']
        
       
        users.loc[j,'TwM_t0'] = users.loc[m].TwM_tCurrent - users.loc[m,'TwM_time']
        users.loc[j,'TwM_tSeed0'] = users.loc[m,'TwM_tSeed0']
        users.loc[j,'TwM_t-1'] = users.loc[m].TwM_tCurrent - users.loc[n,'TwM_time']
        users.loc[j,'TwM_tSeed-1'] = users.loc[n,'TwM_tSeed-1']
        users.loc[j,'TwM_tCurrent'] =  np.ceil(time/30)*30
        
        
        users.loc[j,'Nw_degree0'] = users.loc[m,'Nw_degree']
        users.loc[j,'Nw_degree-1'] = users.loc[n,'Nw_degree']
        users.loc[j,'Nw_inDegree0'] = users.loc[m,'UsM_friendsCount']
        users.loc[j,'Nw_inDegree-1'] = users.loc[n,'UsM_friendsCount']
        users.loc[j,'Nw_outDegree0'] = users.loc[m,'UsM_followersCount']
        users.loc[j,'Nw_outDegree-1'] = users.loc[n,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed0'] = users.loc[m,'Nw_degreeSeed0']
        users.loc[j,'Nw_inDegreeSeed0'] = users.loc[m,'Nw_inDegreeSeed0']
        users.loc[j,'Nw_outDegreeSeed0'] = users.loc[m,'Nw_outDegreeSeed0']
        users.loc[j,'Nw_degreeSeed-1'] = users.loc[n,'Nw_degreeSeed-1']
        users.loc[j,'Nw_inDegreeSeed-1'] = users.loc[n,'Nw_inDegreeSeed-1']
        users.loc[j,'Nw_outDegreeSeed-1'] = users.loc[n,'Nw_outDegreeSeed-1']
        
        
        
        users.loc[j,'SNw_generation0'] = users.loc[m,'SNw_generation']
        users.loc[j,'SNw_generation-1'] = users.loc[n,'SNw_generation']
        users.loc[j,'SNw_timeSinceSeed0'] = users.loc[m,'SNw_timeSinceSeed0']
        users.loc[j,'SNw_timeSinceSeed-1'] = users.loc[n,'SNw_timeSinceSeed-1']
        users.loc[j,'SNw_totalNodesInfected'] = infect_totalNodeInfected(users)
        users.loc[j,'SNw_nodeInfectedCentrality'] = len(sources)/users.loc[j,'SNw_totalNodesInfected']
        users.loc[j,'SNw_totalInDegree'] = infect_totalFriends(users)
        users.loc[j,'SNw_totalOutDegree'] = infect_totalFollowers(users)
        users.loc[j,'SNw_inDegreeCentrality'] = safe_division(in_degree[j], users.loc[j,'SNw_totalInDegree'])
        users.loc[j,'SNw_inDegreeCentrality0'] = users.loc[m,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentrality-1'] = users.loc[n,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality'] = safe_division(out_degree[j], users.loc[j,'SNw_totalOutDegree'])
        users.loc[j,'SNw_outDegreeCentrality0'] = users.loc[m,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality-1'] = users.loc[n,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentralitySeed0'] = users.loc[m,'SNw_inDegreeCentralitySeed0']
        users.loc[j,'SNw_outDegreeCentralitySeed0'] = users.loc[m,'SNw_outDegreeCentralitySeed0']
        users.loc[j,'SNw_inDegreeCentralitySeed-1'] = users.loc[n,'SNw_inDegreeCentralitySeed-1']
        users.loc[j,'SNw_outDegreeCentralitySeed-1'] = users.loc[n,'SNw_outDegreeCentralitySeed-1']


        users.loc[j,'SNw_nFriendsInfected']=len(sources)
        users.loc[j,'SNw_friendsInfectedRatio']=safe_division(len(sources), unique_users.loc[j,'friends_count'])
            
        users.loc[j,'Stat_average_kOut']=round(mean(degreeList), 1)
        users.loc[j,'Stat_average_t']=round(mean(timeList), 1)
        users.loc[j,'Stat_average_deltaDays']=sources_dataframe.user_created_days.mean()
        users.loc[j,'Stat_average_statusesCount']=sources_dataframe.user_statuses_count.mean()
        users.loc[j,'Stat_average_followersCount']=sources_dataframe.followers_count.mean()
        users.loc[j,'Stat_average_favouritesCount']=sources_dataframe.user_favourites_count.mean()
        users.loc[j,'Stat_average_friendsCount']=sources_dataframe.friends_count.mean()
        users.loc[j,'Stat_average_listedCount']=sources_dataframe.user_listed_count.mean()
        users.loc[j,'Stat_average_normalizedUserStatusesCount']=sources_dataframe.normalized_user_statuses_count.mean()
        users.loc[j,'Stat_average_normalizedUserFollowersCount']=sources_dataframe.normalized_user_followers_count.mean()
        users.loc[j,'Stat_average_normalizedUserFavouritesCount']=sources_dataframe.normalized_user_favourites_count.mean()
        users.loc[j,'Stat_average_normalizedUserListedCount']=sources_dataframe.normalized_user_listed_count.mean()
        users.loc[j,'Stat_average_normalizedUserFriendsCount']=sources_dataframe.normalized_user_friends_count.mean()
        if len(degreeList)!=0 :
            users.loc[j,'Stat_max_kOut']=max(degreeList) 
            users.loc[j,'Stat_min_kOut']=min(degreeList) 
        else:
            users.loc[j,'Stat_max_kOut']=''
            users.loc[j,'Stat_min_kOut']=''
#         temp = users[users.index.isin(['j'])]
        return users

In [7]:
# intervals = [60, 30, 15, 7]
intervals = [30]

# Helper function for mean value calculation
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y
    
# infectedNetworkStats = {}
# for interval in intervals:
#     statsDictionary = {}
#     for timestamp in range(0, 24*60+1, interval):
#         infected_dataframe = unique_users[unique_users.time_lapsed <= timestamp]
#         statsDictionary[timestamp] = {}
#         statsDictionary[timestamp]['totalNodeInfected'] = infected_dataframe.shape[0]
#         statsDictionary[timestamp]['totalFollowers'] = sum(infected_dataframe.followers_count)
#         statsDictionary[timestamp]['totalFriends'] = sum(infected_dataframe.friends_count)
#     infectedNetworkStats[interval] = statsDictionary

def infect_totalNodeInfected(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = infected_dataframe.shape[0]
    return states

def infect_totalFollowers(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = sum(infected_dataframe.UsM_followersCount)
    return states

def infect_totalFriends(users):
    infected_dataframe = users[users.TwM_time>=0]
    states = sum(infected_dataframe.UsM_friendsCount)
    return states

in_degree = list(unique_users.friends_count)
out_degree = list(unique_users.followers_count)
degree = [sum(x) for x in zip(in_degree, out_degree)]

In [8]:
network_simulation = pd.DataFrame(columns=["t_current","user_id","source_candidates","infec_candidates",
                                           "followers_count","friends_count","source_id","seed_user_id",
                                           "SIR","generations","time_lapsed","time_since_seed"])
# network_simulation["user"] =unique_users["user"]
network_simulation["user_id"] = unique_users["user_id"]
network_simulation["source_candidates"] = unique_users["source_candidates"]
network_simulation["infec_candidates"] = unique_users["infec_candidates"]
network_simulation["followers_count"] = unique_users["followers_count"]
network_simulation["friends_count"] = unique_users["friends_count"]
#network_simulation.head()
# network_simualtion = load_pickle(path+'/Stanford_network_simulation_0.6.pkl')

In [9]:

infeclen =[len(x) for x in network_simulation['infec_candidates']]
network_simulation['infeclen']=infeclen
network_simulation

,t_current,user_id,source_candidates,infec_candidates,followers_count,friends_count,source_id,seed_user_id,SIR,generations,time_lapsed,time_since_seed,infeclen
0,NaN,214328887,"[105, 62, 147, 148, 42, 124, 32, 10, 96, 18, 1...","[1, 83, 70, 16, 40, 4, 76, 113, 156, 75, 39, 6...",2090,46,NaN,NaN,NaN,NaN,NaN,NaN,147
1,NaN,34428380,"[0, 12, 11, 110, 33, 118, 167, 78, 18, 74, 90,...","[51, 34, 93, 9, 58, 72, 135, 149, 1801, 2947, ...",305257,181,NaN,NaN,NaN,NaN,NaN,NaN,33
2,NaN,17116707,"[18, 56, 52, 111, 153, 97, 38, 35, 31, 169, 20...","[3, 82, 125, 113, 39, 62, 116, 70, 124, 58, 14...",11756,498,NaN,NaN,NaN,NaN,NaN,NaN,77
3,NaN,28465635,"[2, 111, 36, 158, 83, 118, 179, 148, 65, 130, ...","[161, 38, 86, 76, 12, 84, 72, 141, 100, 8, 53,...",57992,1482,NaN,NaN,NaN,NaN,NaN,NaN,91
4,NaN,380580781,"[18, 96, 35, 0, 66, 32, 156, 75, 55, 54, 25, 4...","[5, 87, 51, 140, 33, 22, 72, 58, 47, 10, 83, 6...",15033,219,NaN,NaN,NaN,NaN,NaN,NaN,68
5,NaN,18996905,"[4, 64, 42, 173, 66, 155, 153, 112, 43, 156, 2...","[63, 129, 9, 1, 72, 4170, 14875, 18521, 14885,...",284075,68,NaN,NaN,NaN,NaN,NaN,NaN,32
6,NaN,221036078,"[148, 8, 83, 68, 46, 0, 124, 113, 116, 33, 31,...","[7, 72, 52, 81, 118, 114, 56, 86, 37, 53, 41, ...",2121,206,NaN,NaN,NaN,NaN,NaN,NaN,67
7,NaN,153460275,"[6, 115, 31, 23, 82, 62, 44, 18, 20, 173, 14, ...","[84, 75, 72, 1, 10, 125, 9, 5, 18471, 18503, 1...",4710,35,NaN,NaN,NaN,NaN,NaN,NaN,13
8,NaN,107830991,"[10, 61, 35, 28, 41, 56, 52, 3, 49, 113, 69, 9...","[9, 58, 20, 77, 54, 6, 31, 43, 165, 21, 153, 3...",11627,934,NaN,NaN,NaN,NaN,NaN,NaN,106
9,NaN,17868918,"[8, 87, 123, 20, 37, 16, 66, 95, 80, 111, 69, ...","[51, 39, 81, 93, 58, 122, 72, 144, 4159, 4170,...",16996,552,NaN,NaN,NaN,NaN,NaN,NaN,103


In [10]:
##situation choose:
##a.全部为0   b.从0开始由小到大    c.从0开始由大到小


def seed_chosen():
    index_empty=[]
    for x in range(len(network_simulation)):
        if   math.isnan(network_simulation.loc[x,'generations'])==True :
            if network_simulation.loc[x,'infeclen']<150:
                index_empty.append(x)
    seeds_potential = network_simulation[network_simulation.index.isin(index_empty)]
#     print(seeds_potential)
    seeds_potential  = seeds_potential.sort_values(by='infeclen',ascending=False)
    seeds= seeds_potential.iloc[0:5]
#     print(seeds)
    return seeds

# infeclen =[len(x) for x in seeds['infec_candidates']]
# seeds['infeclen']=infeclen

# seeds = seeds.sort_values(by='infeclen',ascending=False)
# seeds = seeds.iloc[0:5]
# seeds['time_lapsed']=0
# seeds

In [11]:
### seeds ###
def seeds_setting_simulation(seeds,network_simulation):
    for i in range(len(seeds)):
        index = seeds.index.values
        network_simulation.loc[index,'SIR'] = 1
        network_simulation.loc[index,'generations'] = 0
        network_simulation.loc[index,'source_id'] = seeds.loc[index,'user_id']
        network_simulation.loc[index,'time_lapsed'] = seeds.loc[index,'time_lapsed']
        network_simulation.loc[index,'seed_user_id'] = seeds.loc[index,'user_id']
        network_simulation.loc[index,'time_since_seed'] = 0
    return network_simulation

active=20179
normalized_active= 7.211937

def seeds_setting_features(seeds,users):
    for i in range(len(seeds)):
        j = seeds.index.values[i]
#         sources = [x for x in seeds.loc[j,'source_candidates']]
         ###########################################################
        users.loc[j,'UsM_statusesCount'] =active
        users.loc[j,'UsM_normalizedUserStatusesCount']=normalized_active
        ###########################################################
        
        users.loc[j,'UsM_deltaDays0'] = users.loc[j,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount0'] = users.loc[j,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount0']=users.loc[j,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount0']=users.loc[j,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount0']=users.loc[j,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount0']=users.loc[j,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount0']=users.loc[j,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount0']=users.loc[j,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount0']=users.loc[j,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount0']=users.loc[j,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount0']=users.loc[j,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'UsM_deltaDays-1'] = users.loc[j,'UsM_deltaDays']
        users.loc[j,'UsM_statusesCount-1'] = users.loc[j,'UsM_statusesCount']
        users.loc[j,'UsM_followersCount-1']=users.loc[j,'UsM_followersCount']
        users.loc[j,'UsM_favouritesCount-1']=users.loc[j,'UsM_favouritesCount']
        users.loc[j,'UsM_friendsCount-1']=users.loc[j,'UsM_friendsCount']
        users.loc[j,'UsM_listedCount-1']=users.loc[j,'UsM_listedCount']
        users.loc[j,'UsM_normalizedUserStatusesCount-1']=users.loc[j,'UsM_normalizedUserStatusesCount']
        users.loc[j,'UsM_normalizedUserFollowersCount-1']=users.loc[j,'UsM_normalizedUserFollowersCount']
        users.loc[j,'UsM_normalizedUserFavouritesCount-1']=users.loc[j,'UsM_normalizedUserFavouritesCount']
        users.loc[j,'UsM_normalizedUserListedCount-1']=users.loc[j,'UsM_normalizedUserListedCount']
        users.loc[j,'UsM_normalizedUserFriendsCount-1']=users.loc[j,'UsM_normalizedUserFriendsCount']
        
        users.loc[j,'TwM_t0'] =seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_t-1'] =seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_tCurrent'] = np.ceil(seeds.loc[j,'time_lapsed']+0.001/30)*30   
        users.loc[j,'TwM_tSeed0'] = seeds.loc[j,'time_lapsed']                         
        users.loc[j,'TwM_tSeed-1'] = seeds.loc[j,'time_lapsed']
        users.loc[j,'TwM_time'] = seeds.loc[j,'time_lapsed']

        users.loc[j,'Nw_degree0'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_degree-1'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_inDegree0'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_inDegree-1'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_outDegree0'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_outDegree-1'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed0'] = users.loc[j,'Nw_degree']    
        users.loc[j,'Nw_inDegreeSeed0'] = users.loc[j,'UsM_friendsCount'] 
        users.loc[j,'Nw_outDegreeSeed0'] = users.loc[j,'UsM_followersCount']
        users.loc[j,'Nw_degreeSeed-1'] = users.loc[j,'Nw_degree']
        users.loc[j,'Nw_inDegreeSeed-1'] = users.loc[j,'UsM_friendsCount']
        users.loc[j,'Nw_outDegreeSeed-1'] = users.loc[j,'UsM_followersCount']
        
        users.loc[j,'SNw_generation0'] = 0
        users.loc[j,'SNw_generation-1'] = 0
        users.loc[j,'SNw_generation'] = 0
        users.loc[j,'SNw_totalNodesInfected'] = infect_totalNodeInfected(users)
#         users.loc[j,'SNw_nodeInfectedCentrality'] = len(sources)/users.loc[j,'SNw_totalNodesInfected']
        users.loc[j,'SNw_totalInDegree'] = infect_totalFriends(users)
        users.loc[j,'SNw_totalOutDegree'] = infect_totalFollowers(users)
        users.loc[j,'SNw_inDegreeCentrality'] = safe_division(in_degree[j], users.loc[j,'SNw_totalInDegree'])
        users.loc[j,'SNw_inDegreeCentrality0'] = users.loc[j,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentrality-1'] = users.loc[j,'SNw_inDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality'] = safe_division(out_degree[j], users.loc[j,'SNw_totalOutDegree'])
        users.loc[j,'SNw_outDegreeCentrality0'] = users.loc[j,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_outDegreeCentrality-1'] = users.loc[j,'SNw_outDegreeCentrality']
        users.loc[j,'SNw_inDegreeCentralitySeed0'] = users.loc[j,'SNw_inDegreeCentrality']               
        users.loc[j,'SNw_outDegreeCentralitySeed0'] = users.loc[j,'SNw_outDegreeCentrality']                
        users.loc[j,'SNw_inDegreeCentralitySeed-1'] = users.loc[j,'SNw_inDegreeCentrality']                  
        users.loc[j,'SNw_outDegreeCentralitySeed-1'] = users.loc[j,'SNw_outDegreeCentrality']
    return users

In [12]:
# seeds_setting_simulation(seeds,network_simulation)
# seeds_setting_features(seeds,users)


In [13]:
network_simulation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70013 entries, 0 to 70012
Data columns (total 13 columns):
t_current            0 non-null object
user_id              70013 non-null int64
source_candidates    70013 non-null object
infec_candidates     70013 non-null object
followers_count      70013 non-null int64
friends_count        70013 non-null int64
source_id            0 non-null object
seed_user_id         0 non-null object
SIR                  0 non-null object
generations          0 non-null object
time_lapsed          0 non-null object
time_since_seed      0 non-null object
infeclen             70013 non-null int64
dtypes: int64(4), object(9)
memory usage: 6.9+ MB


In [14]:

def simulation(t,unique_users,network_simulation):
    for i in range(len(unique_users)):
        if np.isnan(network_simulation.loc[i,"time_lapsed"]) == False :
            if round(network_simulation.loc[i,"time_lapsed"]) <= t:
                infec_candidates=network_simulation.loc[i,"infec_candidates"]

                for j in range(len(infec_candidates)):
                    infector = network_simulation.loc[i,"infec_candidates"][j]

                    if network_simulation.loc[infector,'SIR']!=1 :
                        user_id1 = network_simulation.loc[i,'user_id']
                        user_id2 = network_simulation.loc[infector,'user_id']
                        processed_user=reprocess_data(t+1,user_id1,user_id2,users,unique_users)

                        valid = processed_user[processed_user.index.isin([infector])]
#                             print(valid['Stat_average_t'])
                        valid = valid.drop(['label','user_id','SNw_generation','TwM_time'],axis=1)

                        valid = valid.astype('float64')

                        columns = list(valid.columns)
                        X = valid[columns]
                        pre_data = xgb.DMatrix(X)
                        infec = model.predict(pre_data)
                        #print(infec)
                        if infec>0.6 :
                            users.loc[infector,'TwM_time'] = t
                            network_simulation.loc[infector,"SIR"] = 1
                            network_simulation.loc[infector,"generations"] = network_simulation.loc[i,"generations"]+1
                            network_simulation.loc[infector,"source_id"] = network_simulation.loc[i,"user_id"] 
                            network_simulation.loc[infector,"seed_user_id"] = network_simulation.loc[i,"seed_user_id"] 
                            network_simulation.loc[infector,"time_lapsed"] = t+1
            #             network_simulation.loc[i,"SIR"]=2 
    return unique_users,network_simulation



In [18]:
##time = 5*60 minutes with resuming, due to releasing memory
seeds_id = list(seeds.index.values)
for t in range(0,6):
    if t < 4 :
        seeds=seed_chosen()
        seeds['time_lapsed']=t
        seeds_setting_simulation(seeds,network_simulation)
        seeds_setting_features(seeds,users)
        for x in range(len(seeds.index.values)):
            seeds_id.append(seeds.index.values[x])
    temp_data=simulation(t,unique_users,network_simulation)
    unique_users = temp_data[0]
    network_simulation = temp_data[1]
    print(t)
    write_to_pickle(path+'/Stanford_network_simulation_Combined.pkl', network_simulation)


KeyboardInterrupt: 

In [16]:
network_simulation["t_current"]=users["TwM_tCurrent"]
# network_simulation

In [17]:
x=[0,1,2,3,4,5]
network_simulation[network_simulation.generations.isin(x)]

,t_current,user_id,source_candidates,infec_candidates,followers_count,friends_count,source_id,seed_user_id,SIR,generations,time_lapsed,time_since_seed,infeclen
0,90,214328887,"[105, 62, 147, 148, 42, 124, 32, 10, 96, 18, 1...","[1, 83, 70, 16, 40, 4, 76, 113, 156, 75, 39, 6...",2090,46,214328887,214328887,1,0,2,0,147
1,30,34428380,"[0, 12, 11, 110, 33, 118, 167, 78, 18, 74, 90,...","[51, 34, 93, 9, 58, 72, 135, 149, 1801, 2947, ...",305257,181,158419434,254839786,1,2,3,NaN,33
2,30,17116707,"[18, 56, 52, 111, 153, 97, 38, 35, 31, 169, 20...","[3, 82, 125, 113, 39, 62, 116, 70, 124, 58, 14...",11756,498,158419434,254839786,1,2,3,NaN,77
3,30,28465635,"[2, 111, 36, 158, 83, 118, 179, 148, 65, 130, ...","[161, 38, 86, 76, 12, 84, 72, 141, 100, 8, 53,...",57992,1482,254839786,254839786,1,1,2,NaN,91
4,30,380580781,"[18, 96, 35, 0, 66, 32, 156, 75, 55, 54, 25, 4...","[5, 87, 51, 140, 33, 22, 72, 58, 47, 10, 83, 6...",15033,219,222261763,222261763,1,1,4,NaN,68
5,30,18996905,"[4, 64, 42, 173, 66, 155, 153, 112, 43, 156, 2...","[63, 129, 9, 1, 72, 4170, 14875, 18521, 14885,...",284075,68,158419434,254839786,1,2,3,NaN,32
6,30,221036078,"[148, 8, 83, 68, 46, 0, 124, 113, 116, 33, 31,...","[7, 72, 52, 81, 118, 114, 56, 86, 37, 53, 41, ...",2121,206,214328887,214328887,1,1,4,NaN,67
7,30,153460275,"[6, 115, 31, 23, 82, 62, 44, 18, 20, 173, 14, ...","[84, 75, 72, 1, 10, 125, 9, 5, 18471, 18503, 1...",4710,35,19933035,115363439,1,3,4,NaN,13
8,30,107830991,"[10, 61, 35, 28, 41, 56, 52, 3, 49, 113, 69, 9...","[9, 58, 20, 77, 54, 6, 31, 43, 165, 21, 153, 3...",11627,934,17116707,254839786,1,3,4,NaN,106
9,30,17868918,"[8, 87, 123, 20, 37, 16, 66, 95, 80, 111, 69, ...","[51, 39, 81, 93, 58, 122, 72, 144, 4159, 4170,...",16996,552,187773078,115363439,1,2,3,NaN,103
